# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [1]:
from collections import defaultdict
import glob
from typing import Tuple

import numpy as np
import xarray as xr
from e3sm_diags.derivations.derivations import DERIVED_VARIABLES


# TODO: Update SET_NAME and SET_DIR
SET_NAME = "area_mean_time_series"
SET_DIR = "662-area-mean-time-series"

DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{SET_DIR}/{SET_NAME}/**"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "/*.nc"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-area-mean-time-series/{SET_NAME}/**"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "/*.nc"))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [2]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_count = 0

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(SET_DIR, "main-area-mean-time-series")

        if fp_dev not in MAIN_GLOB:
            print(f"No production file found to compare with {fp_dev}!")
            missing_count += 1

    for fp_dev in DEV_GLOB:
        fp_main = fp_main.replace("main-area-mean-time-series", SET_DIR)

        if fp_main not in DEV_GLOB:
            print(f"No development file found to compare with {fp_main}!")
            missing_count += 1

    print(f"Number of files missing: {missing_count}")

In [3]:
def _get_relative_diffs():
    # We are mainly focusing on relative tolerance here (in percentage terms).
    atol = 0
    rtol = 1e-5

    for fp_main in MAIN_GLOB:
        if "test.nc" in fp_main or "ref.nc" in fp_main:
            fp_dev = fp_main.replace("main-area-mean-time-series", SET_DIR)

            print("Comparing:")
            print(f"    * {fp_dev}")
            print(f"    * {fp_main}")

            ds1 = xr.open_dataset(fp_dev)
            ds2 = xr.open_dataset(fp_main)

            var_key = fp_main.split("-")[-3]
            # for 3d vars such as T-200
            var_key.isdigit()
            if var_key.isdigit():
                var_key = fp_main.split("-")[-4]

            print(f"    * var_key: {var_key}")

            dev_data = _get_var_data(ds1, var_key)
            main_data = _get_var_data(ds2, var_key)

            if dev_data is None or main_data is None:
                print("    * Could not find variable key in the dataset(s)")
                continue

            try:
                np.testing.assert_allclose(
                    dev_data,
                    main_data,
                    atol=atol,
                    rtol=rtol,
                )
            except (KeyError, AssertionError) as e:
                print(f"    {e}")
            else:
                print(f"    * All close and within relative tolerance ({rtol})")


def _get_var_data(ds: xr.Dataset, var_key: str) -> np.ndarray:
    """Get the variable data using a list of matching keys.

    The `main` branch saves the dataset using the original variable name,
    while the dev branch saves the variable with the derived variable name.
    The dev branch is performing the expected behavior here.

    Parameters
    ----------
    ds : xr.Dataset
        _description_
    var_key : str
        _description_

    Returns
    -------
    np.ndarray
        _description_
    """

    data = None

    var_keys = DERIVED_VARIABLES[var_key].keys()
    var_keys = [var_key] + list(sum(var_keys, ()))

    for key in var_keys:
        if key in ds.data_vars.keys():
            data = ds[key].values
            break

    return data

## 1. Check for matching and equal number of files


In [4]:
_check_if_files_found()

In [5]:
_check_if_matching_filecount()

Matching file count (72 and 72).


In [6]:
_check_if_missing_files()

Number of files missing: 0


## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [20]:
# We are mainly focusing on relative tolerance here (in percentage terms).
atol = 0
rtol = 1e-5

count_mismatch = 0

for filepath_dev in DEV_GLOB:
    if "test.nc" in filepath_dev or "ref.nc" in filepath_dev:
        filepath_main = filepath_dev.replace(SET_DIR, "main-area-mean-time-series")
        print("Comparing:")
        print(filepath_dev, "\n", filepath_main)
        ds1 = xr.open_dataset(filepath_dev)
        ds2 = xr.open_dataset(filepath_main)

        var_key = filepath_dev.split("/")[-2]
        # for 3d vars such as T-200
        var_key.isdigit()
        if var_key.isdigit():
            var_key = filepath_dev.split("-")[-4]

        print("var_key", var_key)
        try:
            np.testing.assert_allclose(
                ds1[var_key].values,
                ds2[var_key].values,
                atol=atol,
                rtol=rtol,
            )
        except AssertionError as e:
            print(e)
            count_mismatch += 1
        else:
            print(f"   * All close and within relative tolerance ({rtol})")

Comparing:
/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/662-area-mean-time-series/area_mean_time_series/FLUT/FLUT-20N50N_test.nc 
 /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-area-mean-time-series/area_mean_time_series/FLUT/FLUT-20N50N_test.nc
var_key FLUT
   * All close and within relative tolerance (1e-05)
Comparing:
/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/662-area-mean-time-series/area_mean_time_series/FLUT/FLUT-20S20N_test.nc 
 /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-area-mean-time-series/area_mean_time_series/FLUT/FLUT-20S20N_test.nc
var_key FLUT
   * All close and within relative tolerance (1e-05)
Comparing:
/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/662-area-mean-time-series/area_mean_time_series/FLUT/FLUT-50N90N_test.nc 
 /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-area-mean-time-series/area_mean_time_series/FLUT/FLUT-50N90N_test.nc
var_key FLUT
   * All close and within relative tolerance (1e-05)
Comparing:
/global/cfs/cdirs/e3sm/www/cd

In [8]:
count_mismatch

72

In [8]:
fp1 = "/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/662-area-mean-time-series/area_mean_time_series/FLUT/FLUT-land_test.nc"
fp2 = "/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-area-mean-time-series/area_mean_time_series/FLUT/FLUT-land_test.nc"

In [9]:
ds_fp1 = xr.open_dataset(fp1)
ds_fp2 = xr.open_dataset(fp2)

In [17]:
ds_fp1

<xarray.Dataset> Size: 160B
Dimensions:  (time: 10)
Coordinates:
  * time     (time) object 80B 0051-01-01 00:00:00 ... 0060-01-01 00:00:00
Data variables:
    FLUT     (time) float64 80B ...

In [12]:
ds_fp2

<xarray.Dataset> Size: 320B
Dimensions:      (time: 10, bound: 2)
Coordinates:
  * time         (time) object 80B 0051-07-02 12:00:00 ... 0060-07-02 12:00:00
Dimensions without coordinates: bound
Data variables:
    bounds_time  (time, bound) object 160B ...
    FLUT         (time) float64 80B ...
Attributes:
    Conventions:  CF-1.0

### Results

All files are within rtol 1e-5, so the changes should be good to go.
